In [1]:
import os
import google.generativeai as genai
import requests
import ast
import re
from PIL import Image

c:\gitProjects\GutEmotions\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

In [3]:
# response = model.generate_content("Please give me python function code to read in a .png image file from a local folder and store it in a variable.")
# print(response.text)

In [4]:
files = genai.list_files()
for f in files:
    print('deleting:', f.name, f.display_name)
    genai.delete_file(f.name)

image_path = "../data/lines/up_down_up_down_up.png"
image_name = image_path[image_path.rindex('/')+1:image_path.rindex('.')]

file_up = genai.upload_file(path=image_path,
                                display_name=image_name)

print(f"Uploaded file '{file_up.display_name}' as: {file_up.uri}")

deleting: files/blq5214gqrcw line_down
Uploaded file 'up_down_up_down_up' as: https://generativelanguage.googleapis.com/v1beta/files/96h7togypldq


In [5]:
image_path[image_path.rindex('/')+1:image_path.rindex('.')]

'up_down_up_down_up'

In [6]:
files = genai.list_files()
for f in files:
    print(f.name, f.display_name)

next(genai.list_files()).name

files/96h7togypldq up_down_up_down_up


'files/96h7togypldq'

In [7]:
first_file = genai.get_file(name=next(genai.list_files()).name)
print(f"Retrieved file '{first_file.display_name}' as: {first_file.uri}")

Retrieved file 'up_down_up_down_up' as: https://generativelanguage.googleapis.com/v1beta/files/96h7togypldq


In [21]:
prompt = "Can you give me an approximate function to describe this line, keeping the y values between -1 and +1?"
prompt = "Can you break this line into 20 pieces and give me approximate x,y coordinates for each piece, with x starting at 0, and with the top of the image equal to 100, and the bottom of the image equal to -100, returning an array that should be assignable to a python variable using: ast.literal_eval()?"
prompt = "Can you break this line into 20 pieces and give me approximate x,y coordinates for each piece, with x starting at 0, and with the vertical center equal to 0 on the y-axis, returning an array that should be assignable to a python variable using: ast.literal_eval()?"

response = model.generate_content([prompt, first_file])

print(response.text)

```python
[
    (0.0, -0.45), (0.5, 0.4), (1.05, -0.45), 
    (1.6, 0.55), (2.15, -0.55), (2.75, 0.65), 
    (3.25, -0.7), (3.8, 0.65), (4.35, -0.65), 
    (4.9, 0.5), (5.5, -0.4), (6.0, 0.3), 
    (6.5, -0.2), (7.05, 0.1), (7.6, 0.0),
    (8.1, 0.2), (8.7, 0.4), (9.2, 0.6),
    (9.7, 0.8), (10.2, 1.0)
]
```


In [22]:
response.text[response.text.index('['):response.text.rindex(']')+1]

'[\n    (0.0, -0.45), (0.5, 0.4), (1.05, -0.45), \n    (1.6, 0.55), (2.15, -0.55), (2.75, 0.65), \n    (3.25, -0.7), (3.8, 0.65), (4.35, -0.65), \n    (4.9, 0.5), (5.5, -0.4), (6.0, 0.3), \n    (6.5, -0.2), (7.05, 0.1), (7.6, 0.0),\n    (8.1, 0.2), (8.7, 0.4), (9.2, 0.6),\n    (9.7, 0.8), (10.2, 1.0)\n]'

In [23]:
arr = ast.literal_eval(response.text[response.text.index('['):response.text.rindex(']')+1])
arr

[(0.0, -0.45),
 (0.5, 0.4),
 (1.05, -0.45),
 (1.6, 0.55),
 (2.15, -0.55),
 (2.75, 0.65),
 (3.25, -0.7),
 (3.8, 0.65),
 (4.35, -0.65),
 (4.9, 0.5),
 (5.5, -0.4),
 (6.0, 0.3),
 (6.5, -0.2),
 (7.05, 0.1),
 (7.6, 0.0),
 (8.1, 0.2),
 (8.7, 0.4),
 (9.2, 0.6),
 (9.7, 0.8),
 (10.2, 1.0)]

In [15]:
arr[:, 1].min()

TypeError: list indices must be integers or slices, not tuple

In [12]:
response = model.generate_content("Please give me a python function to read in a numpy two dimensional array variable, normalizing the second value between -1 and +1.")
print(response.text)

```python
import numpy as np

def normalize_second_value(arr):
  """
  Normalizes the second value of each subarray in a 2D numpy array to the range [-1, 1].

  Args:
    arr: A 2D numpy array.

  Returns:
    A new 2D numpy array with the second value of each subarray normalized.
  """

  # Create a copy to avoid modifying the original array
  normalized_arr = arr.copy()

  # Find the minimum and maximum values of the second column
  min_val = normalized_arr[:, 1].min()
  max_val = normalized_arr[:, 1].max()

  # Normalize the second column
  normalized_arr[:, 1] = 2 * (normalized_arr[:, 1] - min_val) / (max_val - min_val) - 1

  return normalized_arr
```



In [13]:
def normalize_second_value(arr):
  """
  Normalizes the second value of each subarray in a 2D numpy array to the range [-1, 1].

  Args:
    arr: A 2D numpy array.

  Returns:
    A new 2D numpy array with the second value of each subarray normalized.
  """

  # Create a copy to avoid modifying the original array
  normalized_arr = arr.copy()

  # Find the minimum and maximum values of the second column
  min_val = normalized_arr[:, 1].min()
  max_val = normalized_arr[:, 1].max()

  # Normalize the second column
  normalized_arr[:, 1] = 2 * (normalized_arr[:, 1] - min_val) / (max_val - min_val) - 1

  return normalized_arr

In [14]:
normalize_second_value(arr)

TypeError: list indices must be integers or slices, not tuple